In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from efficientnet_pytorch import EfficientNet
from PIL import Image
from tqdm import tqdm

In [3]:
###########################################################
# Define DEFT Module Components
###########################################################

class LocalizationNetwork(nn.Module):
    def __init__(self, input_channels):
        super(LocalizationNetwork, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 8, kernel_size=7)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 10, kernel_size=5)

        # Placeholder fc1 (will update after shape calculation)
        self.fc1 = nn.Linear(1, 32)  # Temporary placeholder
        self.fc2 = nn.Linear(32, 6)  # 6 affine parameters

        # Initialize weights for identity transformation
        self.fc2.weight.data.zero_()
        self.fc2.bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        # Dynamically compute the flattened size
        if not hasattr(self, 'computed_fc1'):
            flattened_size = x.view(x.shape[0], -1).shape[1]
            self.fc1 = nn.Linear(flattened_size, 32).to(x.device)
            self.computed_fc1 = True  # Prevent re-initialization

        x = x.view(x.shape[0], -1)  # Flatten
        x = F.relu(self.fc1(x))
        theta = self.fc2(x)
        theta = theta.view(-1, 2, 3)
        return theta

class WeightingModule(nn.Module):
    def __init__(self, sigma=0.5):
        super(WeightingModule, self).__init__()
        self.lambda_param = nn.Parameter(torch.tensor(0.5))
        self.sigma = sigma

    def forward(self, grid):
        dist2 = grid[..., 0]**2 + grid[..., 1]**2
        weight = 1 + self.lambda_param * torch.exp(-dist2 / (2 * self.sigma ** 2))
        return weight.unsqueeze(-1)

class DEFTModule(nn.Module):
    def __init__(self, input_channels, sigma=0.5):
        super(DEFTModule, self).__init__()
        self.localization = LocalizationNetwork(input_channels)
        self.weighting = WeightingModule(sigma)

    def forward(self, x):
        theta = self.localization(x)
        grid = F.affine_grid(theta, x.size(), align_corners=False)
        weight = self.weighting(grid)
        x_transformed = F.grid_sample(x, grid, align_corners=False)

        if x.shape[1] > 1:
            weight = weight.expand(-1, x.shape[2], x.shape[3], x.shape[1]).permute(0, 3, 1, 2)
        else:
            weight = weight.permute(0, 3, 1, 2)

        x_weighted = x_transformed * weight
        return x_weighted



In [12]:
###########################################################
# Load EfficientNet Model for Feature Extraction
###########################################################
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from efficientnet_pytorch import EfficientNet

# Load EfficientNet (pretrained)
efficientnet_model = EfficientNet.from_pretrained('efficientnet-b0')

# Move model to GPU and set to evaluation mode
efficientnet_model = efficientnet_model.to(device)
efficientnet_model.eval()


# Load DEFT Module
deft_model = DEFTModule(input_channels=3).to(device)  # Input channels = 3 (RGB)
deft_model.eval()

###########################################################
# Define Image Transformation
###########################################################
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


Loaded pretrained weights for efficientnet-b0


In [13]:
###########################################################
# Feature Extraction Function with DEFT
###########################################################
def extract_features(image_path, transform, device=device):
    """Extract EfficientNet features from a single image after DEFT transformation."""
    try:
        image = Image.open(image_path).convert('RGB')
        image = transform(image).unsqueeze(0).to(device)

        # Apply DEFT Transformation
        with torch.no_grad():
            image = deft_model(image)  # Pass through DEFT Module
            
            # EfficientNet requires explicit forward() call
            features = efficientnet_model.extract_features(image).squeeze().cpu().numpy()

        return features
    except Exception as e:
        print(f"Skipping frame: {image_path} due to error: {e}")
        return None

In [14]:
###########################################################
# Paths to RGB and Optical Flow Directories
###########################################################
rgb_path = "D:/Datasets/Datasets/EPIC_Kitchen/RGB/P01_04"
flow_u_path = "D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u"
flow_v_path = "D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v"

label_csv_path = "D:/Datasets/Datasets/EPIC_Kitchen/Label/P01_04.csv"
labels_df = pd.read_csv(label_csv_path)

output_csv = "Features/Feature_P01_04_with_DEFT_EfficeintNet.csv"


In [15]:

###########################################################
# Extract Features with DEFT & Save
###########################################################
S = 5  # Sampling every 5th frame
features_list = []

all_frames = sorted(os.listdir(rgb_path))[::S]

for frame in tqdm(all_frames, desc="Extracting Features with DEFT"):
    rgb_frame_path = os.path.join(rgb_path, frame)
    flow_u_frame_path = os.path.join(flow_u_path, frame)
    flow_v_frame_path = os.path.join(flow_v_path, frame)
    
    rgb_features = extract_features(rgb_frame_path, transform, device)
    flow_u_features = extract_features(flow_u_frame_path, transform, device)
    flow_v_features = extract_features(flow_v_frame_path, transform, device)
    
    if rgb_features is not None and flow_u_features is not None and flow_v_features is not None:
        combined_features = np.concatenate([rgb_features, flow_u_features, flow_v_features])
        
        frame_number = int(frame.split('_')[-1].split('.')[0])
        label_row = labels_df[(labels_df['StartFrame'] <= frame_number) & (labels_df['EndFrame'] >= frame_number)]
        
        if not label_row.empty:
            verb_class = label_row.iloc[0]['Verb_class']
            noun_class = label_row.iloc[0]['Noun_class']
            action_class = label_row.iloc[0]['Action_class']
        else:
            verb_class, noun_class, action_class = 0, 0, 0  # Default if no label found
        
        features_list.append([frame, verb_class, noun_class, action_class] + combined_features.tolist())

if len(features_list) == 0:
    raise ValueError("No valid features extracted. Please check the dataset paths and feature extraction function.")

columns = ["Frame", "Verb_class", "Noun_class", "Action_class"] + [f"Feature_{i}" for i in range(len(combined_features))]
df = pd.DataFrame(features_list, columns=columns)
df.to_csv(output_csv, index=False)

print(f"Feature extraction completed! Saved to {output_csv}")

Extracting Features with DEFT:  50%|████████████████████████▏                       | 635/1262 [01:35<00:47, 13.22it/s]

Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_3161.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_3161.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_3161.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_3161.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_3166.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_3166.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_3166.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_3166.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_3171.jpg due to error: [Errno 2] No such file or directory:

Extracting Features with DEFT:  51%|████████████████████████▍                       | 641/1262 [01:35<00:35, 17.64it/s]

Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_3186.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_3186.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_3186.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_3186.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_3191.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_3191.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_3191.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_3191.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_3196.jpg due to error: [Errno 2] No such file or directory:

Extracting Features with DEFT:  51%|████████████████████████▌                       | 647/1262 [01:35<00:32, 19.06it/s]

...

Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_6251.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_6251.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_6251.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_6251.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_6256.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_6256.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_6256.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_6256.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_6261.jpg due to error: [Errno 2] No such file or directory:

Extracting Features with DEFT: 100%|██████████████████████████████████████████████▉| 1260/1262 [02:06<00:00, 22.51it/s]

Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_6276.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_6276.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_6276.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_6276.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_6281.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_6281.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_6281.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_6281.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_6286.jpg due to error: [Errno 2] No such file or directory:

Extracting Features with DEFT: 100%|███████████████████████████████████████████████| 1262/1262 [02:06<00:00,  9.94it/s]


Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_6301.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_6301.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_6301.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_6301.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_6306.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_6306.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_6306.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_6306.jpg'
Feature extraction completed! Saved to Features/Feature_P01_04_with_DEFT_EfficeintNet.csv


In [17]:
f=pd.read_csv("Features/Feature_P01_04_with_DEFT_EfficeintNet.csv")
f.head()

,Frame,Verb_class,Noun_class,Action_class,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,...,Feature_3830,Feature_3831,Feature_3832,Feature_3833,Feature_3834,Feature_3835,Feature_3836,Feature_3837,Feature_3838,Feature_3839
0,frame_0001.jpg,0,0,0,"[[-0.27609261870384216, -0.24000152945518494, ...","[[-0.27476951479911804, -0.25143420696258545, ...","[[-0.21899963915348053, -0.27313345670700073, ...","[[1.3381232023239136, 1.811214566230774, -0.24...","[[-0.10137911885976791, 0.997386634349823, 0.6...","[[-0.16131477057933807, -0.1400669813156128, -...",...,"[[-0.10623680055141449, -0.046144016087055206,...","[[-0.21481752395629883, -0.21509313583374023, ...","[[-0.2603641152381897, -0.2612024247646332, -0...","[[-0.2783890962600708, -0.22564250230789185, -...","[[0.3416678309440613, -0.030438769608736038, -...","[[-0.2781553268432617, -0.27739906311035156, -...","[[-0.044042691588401794, -0.056324735283851624...","[[-0.09532679617404938, -0.14911344647407532, ...","[[-0.06893198192119598, -0.08556965738534927, ...","[[-0.06898965686559677, -0.23151248693466187, ..."
1,frame_0006.jpg,0,23,0,"[[-0.2593151926994324, -0.27424830198287964, -...","[[-0.2690913677215576, -0.10927192121744156, -...","[[-0.2726871371269226, -0.2476004660129547, -0...","[[0.3025946617126465, 1.39057457447052, -0.196...","[[-0.27489036321640015, 0.6196507215499878, 0....","[[-0.08892244845628738, -0.08882885426282883, ...",...,"[[-0.10592396557331085, -0.06381741911172867, ...","[[-0.22839494049549103, -0.23177215456962585, ...","[[-0.25840067863464355, -0.2517848014831543, -...","[[-0.24809551239013672, -0.20576989650726318, ...","[[0.5891271829605103, 0.33143290877342224, 0.0...","[[-0.2773369550704956, -0.27842792868614197, -...","[[-0.130727156996727, -0.11378748714923859, -0...","[[-0.1443222016096115, -0.20034503936767578, -...","[[-0.15371060371398926, -0.1452828198671341, -...","[[-0.246071919798851, -0.27710291743278503, -0..."
2,frame_0011.jpg,0,23,0,"[[-0.2162872850894928, -0.27826768159866333, -...","[[-0.1949491649866104, -0.25145697593688965, -...","[[-0.2716781198978424, -0.2610386312007904, -0...","[[1.8685377836227417, 2.7821807861328125, -0.2...","[[-0.2756406366825104, 0.4594043493270874, 0.1...","[[-0.04171503707766533, -0.023413926362991333,...",...,"[[-0.09079635143280029, -0.05278758332133293, ...","[[-0.21315327286720276, -0.17192237079143524, ...","[[-0.2725071310997009, -0.2474503517150879, -0...","[[-0.19143331050872803, -0.1720520555973053, -...","[[-0.053092531859874725, 0.00668907118961215, ...","[[-0.2665853500366211, -0.2735857665538788, -0...","[[-0.07383954524993896, -0.0642632246017456, -...","[[-0.1707826405763626, -0.22282925248146057, -...","[[-0.07479698210954666, -0.06356610357761383, ...","[[-0.26254141330718994, -0.2759663462638855, -..."
3,frame_0016.jpg,0,23,0,"[[-0.2771293520927429, -0.22459712624549866, -...","[[-0.10857917368412018, -0.27440589666366577, ...","[[-0.2085351049900055, -0.2739635109901428, -0...","[[2.0774118900299072, 3.2869203090667725, -0.2...","[[-0.27205410599708557, 0.3410997688770294, 0....","[[-0.030833881348371506, -0.02125934325158596,...",...,"[[-0.08019565045833588, -0.06581227481365204, ...","[[-0.18439136445522308, -0.17940396070480347, ...","[[-0.26589855551719666, -0.26821744441986084, ...","[[-0.25372686982154846, -0.195384219288826, -0...","[[1.1503969430923462, 0.8751243352890015, 0.48...","[[-0.276060551404953, -0.26312726736068726, -0...","[[-0.0597376711666584, -0.061445701867341995, ...","[[-0.10821087658405304, -0.20947474241256714, ...","[[-0.053630854934453964, -0.055131249129772186...","[[-0.2035675346851349, -0.2541123330593109, -0..."
4,frame_0021.jpg,0,23,0,"[[-0.2736303508281708, -0.24842281639575958, -...","[[-0.23241643607616425, -0.25125932693481445, ...","[[-0.2677171528339386, -0.2522038519382477, -0...","[[3.0009233951568604, 4.100025177001953, -0.27...","[[-0.19067443907260895, 0.4332495927810669, 0....","[[-0.04865620285272598, -0.0182511024177